In [ ]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from textblob import TextBlob
import re
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import string
import spacy
import os
from textblob import TextBlob

In [ ]:
# Importando os stopwords
stopwords = nltk.corpus.stopwords.words('english')

# Adicionar a palavra ASCO como stopwords
stopwords.append('asco')
stopwords.append('amp')

**<h1>Análise de todos os tweets inclusive os RTs</h1>**

In [ ]:
df = pd.read_csv("tweets_limpos.csv", parse_dates = ["created_at"])

**Limpar novamente os tweets**

In [ ]:
# Função para limpar o texto: substitui todos os caracteres ($,/,@)

def cleanUpText(txt):
    # Remove caracteres
    txt = re.sub(r"\``", '', txt)
    txt = re.sub(r'\$', '', txt)
    txt = re.sub(r'\/', '', txt)
    txt = re.sub(r'\@', '', txt)
    return txt

In [ ]:
# Aplicar a função de limpar os tweets
df['text'] = df['text'].apply(cleanUpText)

**Criar lista de tweets**

In [ ]:
# Criar lista de palavras de todos os tweets
tweets = "" 

# Separar os inisghts em palavras e adicionando na lista 
for i in df["text"]: 
      
    # Transforar em string 
    i = str(i) 
  
    # Separar as palavras 
    tokens = i.split()
        
    # Converter em lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
        
     # Adicionar a lista com espaco entre as palavras
    tweets += " ".join(tokens)+" "

**Tokenization**

In [ ]:
# Separar em palavras (tokenization)
tweets = [word for word in nltk.word_tokenize(tweets)]

**Pos_tag**

In [ ]:
# Verificar os pos_tag
data_tagset = nltk.pos_tag(tweets) 
df_tagset = pd.DataFrame(data_tagset, columns=['Word', 'Tag'])
df_tagset.head()

**Lemmatization de acordo com o pos_tag**

In [ ]:
# Criar o Lemma de cada palavra e criar df_full
lemmatize_text = []

for tweet in tweets:
    output = [tweet, WordNetLemmatizer().lemmatize(tweet, pos='n'), WordNetLemmatizer().lemmatize(tweet, pos='a'),WordNetLemmatizer().lemmatize(tweet, pos='v')]
    lemmatize_text.append(output)
    
# Adicioanr ao df_full com a palavra original e os lemma
df_full = pd.DataFrame(lemmatize_text, columns =['Word', 'Lemmatized Noun', 'Lemmatized Adjective', 'Lemmatized Verb']) 

df_full['Tag'] = df_tagset['Tag']

# Selecionar a palavra lemmatizada de acordo com a pos_tag

df_full['Tempt Lemmatized Word']=df_full['Lemmatized Noun'] + ' | ' + df_full['Lemmatized Adjective']+ ' | ' + df_full['Lemmatized Verb']
lemma_word = df_full['Tempt Lemmatized Word']
tag = df_full['Tag']
i = 0
new_word = []
while i < len(tag):
    words = lemma_word[i].split('|')
    if tag[i] == 'NN' or tag[i] == 'NNS' or tag[i] == 'NNP' or tag[i] == 'NNPS':        
        word = words[0]
    elif tag[i] == 'JJ' or tag[i] == 'JJR' or tag[i] == 'JJS':
        word = words[1]
    elif tag[i] == 'VBG' or tag[i] == 'VBP' or tag[i] == 'VB' or tag[i] == 'VBD' or tag[i] == 'VBN' or tag[i] == 'VBZ':
        word = words[2]
    else:
        word = words[0]
    new_word.append(word)
    i += 1
    
df_full['Lemmatized Word']=new_word
df_full.drop('Tempt Lemmatized Word', axis = 1, inplace=True)
df_full['Lemmatized Word'] = [re.sub(' ', '', x) for x in df_full['Lemmatized Word']]
#
df_full.head()

**Excluir os stopwords**

In [ ]:
# Tirar os stopwords dos tweets lemmatizados e limpos                
lemma_word = df_full['Lemmatized Word']
clean_list_lemma = pd.DataFrame([word for word in lemma_word if not word in stopwords], columns=["Lemmatized Word"])

In [ ]:
# Tirar os stopwords dos tweets como vieram e limpos
text_word = df_full['Word']
clean_list_word = pd.DataFrame([word for word in text_word if not word in stopwords], columns=["Word"])

**Stemmazation**

In [ ]:
tweets_lancaster = [LancasterStemmer().stem(word) for word in tweets]

In [ ]:
tweets_porter = [PorterStemmer().stem(word) for word in tweets]

**Collocations - bigramas e trigramas**

In [ ]:
clean_list_word = [str(x) for x in clean_list_lemma['Lemmatized Word']]

In [ ]:
clean_list_word_text = [x for x in clean_list_word['Word']]

In [ ]:
# Criar um buscador de bigramas nos tokens
buscaGramas = nltk.collocations.FreqDist(clean_list_word)

# Contar quantas vezes cada bigrama aparece nos tokens dos comentários
grama_freq = buscaGramas.most_common(100)

# Converter o dicionário anterior em uma tabela de frequência no formato do Pandas para os bigramas
FreqTabGramas = pd.DataFrame(list(grama_freq), 
                               columns = ['Grama', 'Freq']).sort_values(by = 'Freq', ascending = False)

FreqTabGramas.head(10)

In [ ]:
# Criar um buscador de bigramas nos tokens
buscaBigramas = nltk.collocations.BigramCollocationFinder.from_words(clean_list_word)

# Contar quantas vezes cada bigrama aparece nos tokens dos comentários
bigrama_freq = buscaBigramas.ngram_fd.items()

# Converter o dicionário anterior em uma tabela de frequência no formato do Pandas para os bigramas
FreqTabBigramas = pd.DataFrame(list(bigrama_freq), 
                               columns = ['Bigrama', 'Freq']).sort_values(by = 'Freq', ascending = False)

FreqTabBigramas.head(10)

In [ ]:
# Fazemos o mesmo com trigramas. Fique atento aos métodos que estão sendo usados
buscaTrigramas = nltk.collocations.TrigramCollocationFinder.from_words(clean_list_word)

# Vamos contar quantas vezes cada trigrama aparece nos tokens dos comentários
trigrama_freq = buscaTrigramas.ngram_fd.items()

# Tabela de frequência no formato do Pandas para os trigramas
FreqTabTrigramas = pd.DataFrame(list(trigrama_freq), 
                                columns = ['Trigrama','Freq']).sort_values(by = 'Freq', ascending = False)

FreqTabTrigramas.head(10)

**Remover os substantivos e pronomes**

In [ ]:
# Função para filtrar bigramas ADJ/NN e remover stopwords
def filtra_tipo_token_bigrama(ngram):
    
    # Verifica se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    
    # Loop nos ngramas para verificar se é stopword
    for word in ngram:
        if word in stopwords or word.isspace():
            return False
        
    # Tipos de tokens aceitáveis
    acceptable_types = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Subtipos
    second_type = ('NN', 'NNS', 'NNP', 'NNPS')
    
    # Tags
    tags = nltk.pos_tag(ngram)
    
    # Retorna o que queremos, ADJ/NN
    if tags[0][1] in acceptable_types and tags[1][1] in second_type:
        return True
    else:
        return False

In [ ]:
# Agora filtramos os bigramas
bigramas_filtrados = FreqTabBigramas[FreqTabBigramas.Bigrama.map(lambda x: filtra_tipo_token_bigrama(x))]
# Visualiza a tabela
bigramas_filtrados.head(10)

In [ ]:
# Função para filtrar trigramas ADJ/NN e remover stopwords
def filtra_tipo_token_trigrama(ngram):
    
    # Verifica se é pronome
    if '-pron-' in ngram or 't' in ngram:
        return False
    
    # Loop nos ngramas para verificar se é stopword
    for word in ngram:
        if word in stopwords or word.isspace():
            return False
        
    # Tipos de tokens aceitáveis
    first_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Subtipos
    second_type = ('JJ', 'JJR', 'JJS', 'NN', 'NNS', 'NNP', 'NNPS')
    
    # Tags
    tags = nltk.pos_tag(ngram)
    
    # Retorna o que queremos, ADJ/NN
    if tags[0][1] in first_type and tags[2][1] in second_type:
        return True
    else:
        return False

In [ ]:
# Agora filtramos os trigramas
trigramas_filtrados = FreqTabTrigramas[FreqTabTrigramas.Trigrama.map(lambda x: filtra_tipo_token_trigrama(x))]
# Visualiza a tabela
trigramas_filtrados.head(10)

In [ ]:
# Métricas de associação de bigramas (esse objeto possui diversos atributos, como freq, pmi, teste t, etc...)
bigramas = nltk.collocations.BigramAssocMeasures()

# Criamos a tabela
PMITabBigramas = pd.DataFrame(list(buscaBigramas.score_ngrams(bigramas.pmi)), 
                              columns = ['Bigrama', 'PMI']).sort_values(by = 'PMI', ascending = False)
# Visualiza a tabela
PMITabBigramas.head(5)

In [ ]:
# Métricas de associação de trigramas
trigramas = nltk.collocations.TrigramAssocMeasures()

# Criamos a tabela
PMITabTrigramas = pd.DataFrame(list(buscaTrigramas.score_ngrams(trigramas.pmi)), 
                               columns = ['Trigrama', 'PMI']).sort_values(by = 'PMI', ascending = False)
# Visualiza a tabela
PMITabTrigramas.head(5)

In [ ]:
# Criamos a tabela para os bigramas
# Observe como o resultado do teste t é obtido: buscaBigramas.score_ngrams(bigramas.student_t)
TestetTabBigramas = pd.DataFrame(list(buscaBigramas.score_ngrams(bigramas.student_t)), 
                             columns = ['Bigrama', 'Teste-t']).sort_values(by = 'Teste-t', ascending = False)
# Vamos aplicar o filtro pelo tipo de token conforme aplicamos no método 1
bigramas_t_filtrados = TestetTabBigramas[TestetTabBigramas.Bigrama.map(lambda x: filtra_tipo_token_bigrama(x))]
# Visualiza a tabela
bigramas_t_filtrados.head(10)

In [ ]:
# Criamos a tabela para os trigramas
TestetTabTrigramas = pd.DataFrame(list(buscaTrigramas.score_ngrams(trigramas.student_t)), 
                                  columns = ['Trigrama', 'Teste-t']).sort_values(by = 'Teste-t', ascending = False)
# Vamos aplicar o filtro pelo tipo de token conforme aplicamos no método 1
trigramas_t_filtrados = TestetTabTrigramas[TestetTabTrigramas.Trigrama.map(lambda x: filtra_tipo_token_trigrama(x))]
# Visualiza a tabela
trigramas_t_filtrados.head(10)

In [ ]:
# Prepara a tabela
# Observe como estamos coletando a estatística qui-quadrado: buscaBigramas.score_ngrams(bigramas.chi_sq)
QuiTabBigramas = pd.DataFrame(list(buscaBigramas.score_ngrams(bigramas.chi_sq)), 
                              columns = ['Bigrama', 'Qui']).sort_values(by = 'Qui', ascending = False)
# Visualiza a tabela
QuiTabBigramas.head(5)

In [ ]:
# Prepara a tabela
QuiTabTrigramas = pd.DataFrame(list(buscaTrigramas.score_ngrams(trigramas.chi_sq)), 
                               columns = ['Trigrama', 'Qui']).sort_values(by = 'Qui', ascending = False)
# Visualiza a tabela
QuiTabTrigramas.head(5)

**Tabelar os resultados**

In [ ]:
# Vamos extrair os 10 Collocations bigramas mais relevantes de acordo com cada um dos 4 métodos usados
# Lembre-se que aplicamos filtros para remover as stopwords e devemos usar a tabela filtrada
metodo1_bigrama = bigramas_filtrados[:10].Bigrama.values
metodo2_bigrama = PMITabBigramas[:10].Bigrama.values
metodo3_bigrama = bigramas_t_filtrados[:10].Bigrama.values
metodo4_bigrama = QuiTabBigramas[:10].Bigrama.values

In [ ]:
# Vamos extrair os 10 Collocations trigramas mais relevantes de acordo com cada um dos 4 métodos usados
# Lembre-se que aplicamos filtros para remover as stopwords e devemos usar a tabela filtrada
metodo1_trigrama = trigramas_filtrados[:10].Trigrama.values
metodo2_trigrama = PMITabTrigramas[:10].Trigrama.values
metodo3_trigrama = trigramas_t_filtrados[:10].Trigrama.values
metodo4_trigrama = QuiTabTrigramas[:10].Trigrama.values

In [ ]:
# Vamos criar um super dataframe com todos os resultados para bigramas
comparaBigramas = pd.DataFrame([metodo1_bigrama, metodo2_bigrama, metodo3_bigrama, metodo4_bigrama]).T

In [ ]:
# Nossa tabela precisa de nomes para as colunas
comparaBigramas.columns = ['Frequência', 
                           'PMI', 
                           'Teste-t', 
                           'Teste Qui-quadrado']

In [ ]:
# Visualiza a tabela - Padrão CSS
comparaBigramas.style.set_properties(**{'background-color': 'green', 
                                        'color': 'white', 
                                        'border-color': 'white'})

In [ ]:
# Vamos criar um super dataframe com todos os resultados para trigramas
comparaTrigramas = pd.DataFrame([metodo1_trigrama, metodo2_trigrama, metodo3_trigrama, metodo4_trigrama]).T

In [ ]:
# Nossa tabela precisa de nomes para as colunas
comparaTrigramas.columns = ['Frequência', 
                            'PMI', 
                            'Teste-t', 
                            'Teste Qui-quadrado']

In [ ]:
# Visualiza a tabela
comparaTrigramas.style.set_properties(**{'background-color': 'blue', 
                                         'color': 'white', 
                                         'border-color': 'white'})

**Wordcloud**

In [ ]:
wordcloud_text = str(clean_list_word)
wordcloud_text = re.sub("'", '', wordcloud_text)

In [ ]:
# Setando a wordcloud por palavras
wordcloud = WordCloud(width = 3000, height = 2000, 
                background_color ='black', 
                collocations=True,
                stopwords=stopwords,
                colormap='Set2',
                min_font_size = 10).generate(wordcloud_text)

In [ ]:
# plotando a wordcloud
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 

In [ ]:
# Setando a wordcloud por palavras
wordcloud = WordCloud(width = 3000, height = 2000, 
                background_color ='white', 
                collocations=False,
                stopwords=stopwords,
                colormap='Set2',
                min_font_size = 10).generate(wordcloud_text)

In [ ]:
# plotando a wordcloud
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 

**Grafico com número de tweets por dia com 'asco'**

In [ ]:
df[["created_at", "text"]].groupby("created_at").count().plot(title="Number of Tweets", figsize=(10,6), grid=True)

In [ ]:
df[["created_at", "text"]].groupby("created_at").count()

**<h1>Análise de sentimentos</h1>**

In [ ]:
# Definindo a função de avaliar os sentimentos (objetividade e polaridade)
def getTextSubjectivity(txt):
    return TextBlob(txt).sentiment.subjectivity

def getTextPolarity(txt):
    return TextBlob(txt).sentiment.polarity

In [ ]:
# Aplicando a função nos insights
df['Subjectivity'] = df['text'].apply(getTextSubjectivity)
df['Polarity'] = df['text'].apply(getTextPolarity)

In [ ]:
# Eliminando linha vazia caso exista
df = df.drop(df[df['text'] == ''].index)

In [ ]:
# Definindo a função para categorizar os sentimentos como negativo, neutro ou positivo
def getTextAnalysis(a):
    if a < 0:
        return "Negative"
    elif a == 0:
        return "Neutral"
    else:
        return "Positive"

In [ ]:
# Aplicando a função para categorizar
df['Score'] = df['Polarity'].apply(getTextAnalysis)

In [ ]:
df.head()

In [ ]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
df['neg'] = df['text'].apply(lambda x:analyzer.polarity_scores(x)['neg'])
df['neu'] = df['text'].apply(lambda x:analyzer.polarity_scores(x)['neu'])
df['pos'] = df['text'].apply(lambda x:analyzer.polarity_scores(x)['pos'])
df['compound'] = df['text'].apply(lambda x:analyzer.polarity_scores(x)['compound'])
df['Score'] = df['compound'].apply(getTextAnalysis)
df.head()

In [ ]:
# Plotando grafico com count de insights
labels = df.groupby('Score').count().index.values

values = df.groupby('Score').size().values

plt.bar(labels, values)

In [ ]:
values.tolist()

In [ ]:
# Plotando grafico com pct de insights
positive = df[df['Score'] == 'Positive']
negative = df[df['Score'] == 'Negative']
neutral = df[df['Score'] == 'Neutral']

positive_pct = (positive.shape[0]/(df.shape[0])*100)
negative_pct = (negative.shape[0]/(df.shape[0])*100)
neutral_pct = (neutral.shape[0]/(df.shape[0])*100)

labels2 = df.groupby('Score').count().index.values

values2 = [negative_pct, neutral_pct, positive_pct]

plt.bar(labels2, values2)
plt.ylim(0, 100)

In [ ]:
# Plotando grafico de dispersão de insights positivos/negativos/neutros
for index, row in df.iterrows():
    if row['Score'] == 'Positive':
        plt.scatter(row['Polarity'], row['Subjectivity'], color="green")
    elif row['Score'] == 'Negative':
        plt.scatter(row['Polarity'], row['Subjectivity'], color="red")
    elif row['Score'] == 'Neutral':
        plt.scatter(row['Polarity'], row['Subjectivity'], color="blue")

plt.title('Twitter Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
# add legend
plt.show()